# Recommendation systems
Welcome to your first homework assignment about recommendation systems.


## The Cold Start Problem 

The colaborative filtering method discussed in class does not address the problem of new user or new movies. What prediction would you use in these cases:

* A new user but a known movie
* A new movie and a known user
* A new user and new movie

In [976]:
import numpy as np
import pandas as pd

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [977]:
# The first row says that user 11 reated movie 1 with a score of 4
!cat tiny_training2.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [1019]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [979]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    # YOUR CODE HERE
    _,df['userId'],num_users = proc_col(df['userId'])
    _,df['movieId'],num_movies = proc_col(df['movieId'])

    return df, num_users, num_movies

In [980]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)

In [981]:
df

,userId,movieId,rating
0,0,0,4
1,0,1,5
2,1,1,5
3,1,2,3
4,2,0,4
5,2,1,4
6,3,0,5
7,3,3,2
8,4,0,1
9,4,3,4


In [982]:
assert(num_users == 7)

In [983]:
assert(num_movies == 4)

In [984]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

## Initializing parameters

In [985]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[3.55790894, 4.69774849, 0.92361109, 1.58739544, 3.00593239],
       [4.69774849, 7.44656163, 1.18135616, 2.64524868, 4.74559066],
       [0.92361109, 1.18135616, 0.24548062, 0.34025121, 0.69616965],
       [1.58739544, 2.64524868, 0.34025121, 1.61561   , 2.41361975],
       [3.00593239, 4.74559066, 0.69616965, 2.41361975, 3.82505541],
       [2.02000808, 3.29656257, 0.43174569, 2.065911  , 3.07264619],
       [2.07691001, 3.02887291, 0.53270924, 1.02482544, 1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [986]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [987]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [988]:
print(Y)

  (0, 0)	4
  (2, 0)	4
  (3, 0)	5
  (4, 0)	1
  (0, 1)	5
  (1, 1)	5
  (2, 1)	4
  (6, 1)	1
  (1, 2)	3
  (3, 3)	2
  (4, 3)	4
  (5, 3)	5
  (6, 3)	3


## Predicting Ratings

In [989]:
def predict(df, emb_user, emb_movie):
    """ This function computes df["prediction"] without doing (U*V^T).
    
    Compute df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """    
    users = df['userId']
    movies = df['movieId']
    df['prediction'] = (emb_user[users] * emb_movie[movies]).sum(axis=1)
    return df

In [990]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
df = predict(df, emb_user, emb_movie)
assert(df["prediction"][12] == 3.0)

In [991]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
df = predict(df, emb_user, emb_movie)
pred_12 = df["prediction"][12]
assert(np.around(pred_12, decimals=2)== 2.05)

## Calculating the cost function

In [992]:

def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction using the predict function.
    Prediction for user i and movie j is emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    prediction = predict(df, emb_user, emb_movie)['prediction']
    actual = df['rating']
    error = np.mean(np.power(actual-prediction,2))
    return error

In [993]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

In [994]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 4.36)

## Calculating gradient

In [995]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [996]:
def gradient(df, emb_user, emb_movie):
    """ Computes the gradient.
    
    Hint: First compute df["prediction"]. Then use df2matrix
    to get a sparse matrix Y and Y_hat.
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    users = df['userId']
    movies = df['movieId']
    df['prediction'] = predict(df, emb_user, emb_movie)['prediction']
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    Y_hat = df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="prediction")
    N = df.shape[0]
    nambla = (Y.todense()-Y_hat)
    d_emb_user = (-2/N) * np.dot(nambla,emb_movie)
    d_emb_movie = (-2/N) * np.dot(nambla.T,emb_user)

    return d_emb_user, d_emb_movie

In [997]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
grad_user, grad_movie = gradient(df, emb_user, emb_movie)

In [998]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [999]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Using gradient descent with momentum

In [1000]:
# you can use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    momentum = 0.9
    update_user, update_movie = gradient(df, emb_user, emb_movie)
    
    for iteration in range(iterations):
        #calculate gradients
        #updates
        if iteration > 0:
            du, dm = gradient(df, emb_user, emb_movie)
            update_user =  update_user * (momentum) + du * (1 - momentum)
            update_movie = update_movie * (momentum) + dm * (1 - momentum)
        
        #parameter changes
        emb_user -= learning_rate * update_user
        emb_movie -= learning_rate *update_movie
        if iteration%50 == 49:
            print('Training Error Rate: '+str(cost(df, emb_user  , emb_movie)))
            if df_val is not None:
                print('Validation Error Rate: '+str(cost(df_val, emb_user, emb_movie)))
    return emb_user, emb_movie

In [1001]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

Training Error Rate: 1.701364306188372
Training Error Rate: 0.9748705172056423
Training Error Rate: 0.6991455656925724
Training Error Rate: 0.5265621094145297


In [1002]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.53)

## Predicting on new data

In [1299]:
# here is a handy function from fast.ai
def new_proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [1333]:

def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    # make sure have right indeces
    df_val.set_index(['userId', 'movieId'])
    df_train.set_index(['userId', 'movieId'])
    # validations which are in training
    distinct_users = df_train['userId'].unique()
    distinct_movies = df_train['movieId'].unique()
    index = df_val.userId.isin(distinct_users) & df_val.movieId.isin(distinct_movies) 
    # trainings that are in validation
    distinct_users2 = df_val['userId'].unique()
    distinct_movies2 = df_val['movieId'].unique()
    df_val    = df_val[index]
    length = len(df_val)
    new_df = pd.concat([df_train,df_val])
    encoded, num_users, num_movies = encode_data(new_df)
    new_val = encoded.iloc[-length:]
    return new_val

In [1334]:
df_t = pd.read_csv("tiny_training2.csv")
df_v = pd.read_csv("tiny_val2.csv")
df_v = encode_new_data(df_v, df_t)


In [1335]:
assert(len(df_v.userId.unique())==2)

In [1336]:
assert(len(df_v) == 2)

## Putting it all together


In [1337]:
# Don't change this path use a simlink if you have the data somewhere else
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())

print(len(val), len(df_val))

20386 19591


In [1338]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=200, learning_rate=1, df_val=df_val)

Training Error Rate: 9.48094555948595
Validation Error Rate: 9.60843885425874
Training Error Rate: 6.6913902077377765
Validation Error Rate: 6.824174774852403
Training Error Rate: 4.803840866998358
Validation Error Rate: 4.9284672300475245
Training Error Rate: 3.7567143651597688
Validation Error Rate: 3.868546473763612


In [1339]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

3.7567143651597688 3.868546473763612


Please check that when you run gradient descent for 2000 iterations. `val_mse` should be around 0.855

In [1340]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=1) == 3.8)

In [1341]:
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000, learning_rate=1, df_val=df_val)

20386 19591
Training Error Rate: 9.48094555948595
Validation Error Rate: 9.60843885425874
Training Error Rate: 6.6913902077377765
Validation Error Rate: 6.824174774852403
Training Error Rate: 4.803840866998358
Validation Error Rate: 4.9284672300475245
Training Error Rate: 3.7567143651597688
Validation Error Rate: 3.868546473763612
Training Error Rate: 3.0962612272822643
Validation Error Rate: 3.1939773867373824
Training Error Rate: 2.640211866512512
Validation Error Rate: 2.727822226645782
Training Error Rate: 2.3096501987094094
Validation Error Rate: 2.3910847231805987
Training Error Rate: 2.061108409880841
Validation Error Rate: 2.138973908934579
Training Error Rate: 1.8684796472379896
Validation Error Rate: 1.9444189915928134
Training Error Rate: 1.7153418975229036
Validation Error Rate: 1.7904100562562837
Training Error Rate: 1.5909703181142627
Validation Error Rate: 1.665863604254957
Training Error Rate: 1.488121595983667
Validation Error Rate: 1.5633109911440302
Training Error Ra

In [1342]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

0.7377380605563745 0.8558080144508536
